In [18]:
import yfinance as yf
import pandas as pd
import datetime as dt
import requests as req
from bs4 import BeautifulSoup
import utils


In [ ]:
tickers = ["GNFC"]

In [87]:
# financials-table-container
url = "https://www.tickertape.in/stocks/tata-consultancy-services-TCS/financials?period=annual&statement=cashflow"
response = req.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
column = soup.findAll("div", {"data-row": "cafFcf"})
financial_year = utils.getFinancialYear()
financial_year_limit = financial_year - 9
free_cash_flows = []


for i, x in enumerate(column):
    if i == 0:
        continue

    data = BeautifulSoup(str(x), 'html.parser').get_text().replace(',','')
    

    if (len(data) > 0):
        free_cash_flows.append(float(data))

print(free_cash_flows)



[5120.37, 8977.11, 11625.26, 16419.57, 17122.0, 23234.0, 23205.0, 26362.0, 29120.0]


In [19]:
# Get Cashflows
apv_api_key = '2N8UEPVRC2AWTK6B'

cashflows = req.get("https://www.alphavantage.co/query?function=CASH_FLOW&symbol=TCS&apikey=2N8UEPVRC2AWTK6B")
cashflows = cashflows.json()
annual_cf_df = pd.DataFrame(cashflows['annualReports']).sort_index(ascending=False)
dates = annual_cf_df['fiscalDateEnding'].values.tolist()
years = []

for date in dates:
    years.append(dt.datetime.strptime(date, "%Y-%m-%d").year)

In [57]:
fcf_df = pd.DataFrame()
fcf_df["Year"] = years
fcf_df["Operating Cashflow"] = annual_cf_df["operatingCashflow"].astype(int)
fcf_df["Cap Ex"] = annual_cf_df["capitalExpenditures"].astype(int)
fcf_df["FCF"] = fcf_df["Operating Cashflow"] - fcf_df["Cap Ex"]
growth_rate = [0]

for i in range(len(fcf_df["FCF"])):
    if i < len(fcf_df["FCF"]) - 1:
        pct_diff = (fcf_df["FCF"].iloc[i+1] - fcf_df["FCF"].iloc[i])/abs(fcf_df["FCF"].iloc[i])
        growth_rate.append(pct_diff * 100)

fcf_df["Growth Rate"]  = growth_rate
avg_growth = fcf_df["Growth Rate"].iloc[1:].mean()

avg_growth

-35.630076966509364